![head.png](figures/head.jpg)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

In [2]:
# Packages used in this notebook
import pandas as pd
from sqlalchemy import create_engine
import requests
import json

# Object oriented programming in Python

[Object-Oriented Programming (OOP)](https://en.wikipedia.org/wiki/Object-oriented_programming) is a coding paradigm that represents **real-world entities as "objects"** in programming. These objects have **attributes (data)** and **behavior (methods)**, which can be manipulated to solve complex problems. Python, with its clear syntax and powerful in-built tools, is a popular language for implementing OOP.

See:
* https://docs.python.org/3/tutorial/classes.html

In the context of financial economics, OOP can be particularly useful due to its abstraction and modularity. Financial instruments, for instance, can be modeled as objects. A **bond** could be an object with attributes such as **coupon rate, face value, maturity date** and methods to calculate **present value, yield to maturity, and risk measures**.

A **class, which is a blueprint for creating objects**, could be designed for a bond. From this class, you can create multiple bond objects, each representing a specific bond in the market.

Here's a basic example:

In [2]:
class Bond:
    def __init__(self, face_value, coupon_rate, maturity_date):
        self.face_value = face_value
        self.coupon_rate = coupon_rate
        self.maturity_date = maturity_date

    def calculate_present_value(self, discount_rate):
        # Implementation here
        pass

# Creating a bond object
bond1 = Bond(1000, 0.05, '2030-12-31')

Inheritance, another key feature of OOP, allows classes to inherit attributes and methods from other classes. This can represent financial concepts like hierarchies of asset classes. For example, a Bond class could inherit from a more general Security class, as all bonds are securities but have additional unique properties.

Through encapsulation, OOP allows data and methods to be bundled together, keeping code clean, reusable, and easy to maintain. Polymorphism, another crucial concept in OOP, allows methods to perform differently based on the object calling them. This is useful when dealing with a variety of financial instruments that might share similar methods but require different implementations. For instance, the method to calculate yield might differ between a bond and a stock, despite the shared name.

Let's extend the earlier example with inheritance and polymorphism:

In [3]:
class Security:  # This is a class
    def __init__(self, ticker):
        self.ticker = ticker
        

class Bond(Security):  # This is now a subclass of Security
    def __init__(self, ticker, coupon_rate, maturity_date):
        super().__init__(ticker)  # This invokes the init of the upstream class
        self.coupon_rate = coupon_rate
        self.maturity_date = maturity_date

    def calculate_present_value(self):
        # Implementation specific to bond
        pass

    def calculate_duration(self):
        # Implementation specific to bond
        pass

class Stock(Security):  # This is another subclass
    def __init__(self, ticker, dividends):
        super().__init__(ticker)  # This invokes the init of the upstream class
        self.dividends = dividends

    def calculate_present_value(self):
        # Implementation specific to stock
        pass

    def calculate_dividend_yield(self):
        # Implementation specific to stock
        pass

# Creating objects
bond1 = Bond(1000, 0.05, '2030-12-31')
stock1 = Stock(50, 2)

# Calculating yield
bond1_pv = bond1.calculate_present_value()
stock1_pv = stock1.calculate_present_value()

# Returning attributes, note the missing ()
bond1_coupon = bond1.coupon_rate

# Loose coupling

Loose coupling is a design principle in programming that advocates for **minimal interdependencies between modules or components**. This design makes individual modules more reusable and changes in one module less likely to impact others. In Python, and specifically in the context of financial economics, loose coupling can be achieved through various strategies, including object-oriented programming, interfaces, dependency injection, and the use of events or callbacks.

Suppose we have a **trading system consisting of multiple components, such as a market data handler, a trading strategy, and an execution handler**. In a tightly-coupled system, these components would directly interact with each other, leading to high dependency. Any change in one component could necessitate changes in others. However, with loose coupling, we aim to minimize these dependencies.


## Loose coupling using `class` objects

Let's demonstrate loose coupling in the example of a trading strategy:

In [4]:
class MarketDataHandler:
    def get_price(self, ticker):
        # Returns market price for the given ticker
        pass

class ExecutionHandler:
    def execute_order(self, ticker, quantity, order_type):
        # Executes the order in the market
        pass

class TradingStrategy:
    def __init__(self, data_handler: MarketDataHandler, execution_handler: ExecutionHandler):
        self.data_handler = data_handler
        self.execution_handler = execution_handler

    def execute_trade(self, ticker):
        price = self.data_handler.get_price(ticker)
        if self.is_favorable_price(price):
            self.execution_handler.execute_order(ticker, 100, 'buy')

    def is_favorable_price(self, price):
        # Decides whether the price is favorable for buying (boolean)
        pass

In this example, **TradingStrategy doesn't directly depend on the specific implementations of MarketDataHandler or ExecutionHandler**. Instead, it **depends on the interfaces**, i.e., the methods these classes are supposed to implement. This is a simple example of **Dependency Injection**, where the dependencies (MarketDataHandler and ExecutionHandler) are "injected" into the dependent class (TradingStrategy).

This makes the system loosely coupled as we can switch out the market data source or the execution mechanism without modifying the TradingStrategy class. For instance, you could replace a real-time market data handler with a historical one for backtesting purposes, or replace a simulated execution handler with a live one for actual trading, all without changing the trading strategy code.

## Loose coupling using a data base

Loose coupling can also be achieved with a **central data base that connects code components**. This is particularly useful when managing data access, running programs on different servers or in different frequencies. Please note that this type of coupling might require **simultaneous read/write operations that require some form of [concurrency control](https://en.wikipedia.org/wiki/Concurrency_control)**. Robust concurrency for larger projects often requires a server based solution, such as [Amazon Relational Database Service (AWS-RDS)](https://aws.amazon.com/rds/).

Consider a modification of the example above, where we couple the core functions through some "database_connection":

In [5]:
class MarketDataHandler:
    def __init__(self, database_connection):
        self.database_connection = database_connection

    def get_data(self, ticker):
        # Returns market price for the given ticker
        pass

    def save_market_data(self, ticker):
        # Saves prices to database
        pass

class TradingStrategy:
    def __init__(self, database_connection):
        self.database_connection = database_connection

    def load_market_data(self, ticker):
        # Load prices 
        pass

    def save_trade_signal(self, ticker):
        # Saves trade signal for ticker to database 
        pass

class ExecutionHandler:
    def __init__(self, database_connection):
        self.database_connection = database_connection
        
    def execute_order(self, ticker, quantity, order_type):
        # Trade the order in the market
        pass
    
    def load_trade_signal(self):
        # Load trading signal from db 
        pass
    
    def execute_trade_signal(self, ticker):
        # Execute trade signal for ticker
        if self.load_trade_signal(ticker):
            self.execute_order(ticker, 100, 'buy')


In this example we have separated three essential tasks, namely:
* download market data,
* generate a trading signal, and
* execute the signal,

that connect through a central database. We will pick this up again below, with examples on the collection and storage of data.

# Application Programming Interfaces (APIs)

Application Programming Interfaces (APIs) allow different software applications to communicate and share data, offering a structured way to extract data from online resources. Here, we will demonstrate how to retrieve public market data from Deribit's API using Python. You could think of APIs as a tool that allows us to couple external software and data into our workflow.

Deribit is a cryptocurrency exchange platform that offers a public API for accessing its market data. This data includes information about the current and historic state of the market, trades, and more.

In this guide, we'll be making use of Python's requests library, a simple yet powerful HTTP library. Other market data providers might come with their individual python packages, such as Refinitiv Eikon or Bloomberg. Many APIs require authentication as they are payed services or link to an individual user. An example for the latter is Deribit's trading API, which allows us to place trades after authentication.

See:
* https://docs.deribit.com/

In [6]:
# Define the URL of the endpoint
url = "https://www.deribit.com/api/v2/public/get_book_summary_by_currency"

# Define the parameters
params = {
    "currency": "BTC",  # Cryptocurrency to fetch data for
    "kind": "future"    # Type of data to fetch
}

# Send the GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the response to JSON
    print(json.dumps(data, indent=4))  # Print the data
else:
    print(f"Failed to retrieve data: {response.status_code}")


{
    "jsonrpc": "2.0",
    "result": [
        {
            "mid_price": 66973.75,
            "volume_notional": 7111840.0,
            "volume_usd": 7111840.0,
            "quote_currency": "USD",
            "estimated_delivery_price": 66436.33,
            "creation_timestamp": 1713902486446,
            "base_currency": "BTC",
            "volume": 105.88862586,
            "price_change": -0.1192,
            "open_interest": 60707710,
            "ask_price": 66982.5,
            "bid_price": 66965.0,
            "instrument_name": "BTC-31MAY24",
            "mark_price": 66980.6,
            "last": 67032.5,
            "low": 66497.5,
            "high": 67865.0
        },
        {
            "mid_price": 66414.25,
            "funding_8h": -1.116e-05,
            "current_funding": 0.0,
            "volume_notional": 368904940.0,
            "volume_usd": 368904940.0,
            "quote_currency": "USD",
            "estimated_delivery_price": 66436.33,
            "creat

This script fetches the summary of the BTC future book from Deribit's API. The response data is then parsed to JSON and printed. For details on the data see: https://docs.deribit.com/#public-get_book_summary_by_currency

Remember to always handle API responses properly, as they may not always be successful. In this example, we check if the status code is 200 (indicating success), before proceeding to handle the data. If the status code is not 200, an error message is printed instead.

To work with the data, we can use a familiar Python package:

In [7]:
market_data = pd.DataFrame(data["result"])
display(market_data)

,mid_price,volume_notional,volume_usd,quote_currency,estimated_delivery_price,creation_timestamp,base_currency,volume,price_change,open_interest,ask_price,bid_price,instrument_name,mark_price,last,low,high,funding_8h,current_funding
0,66973.75,7111840.0,7111840.0,USD,66436.33,1713902486446,BTC,105.888626,-0.1192,60707710,66982.5,66965.0,BTC-31MAY24,66980.60,67032.5,66497.5,67865.0,NaN,NaN
1,66414.25,368904940.0,368904940.0,USD,66436.33,1713902486446,BTC,5544.539464,-0.1691,775504490,66414.5,66414.0,BTC-PERPETUAL,66422.22,66414.0,65827.0,67268.0,-0.000011,0.0
2,67571.25,27468090.0,27468090.0,USD,66436.33,1713902486446,BTC,405.072827,-0.1551,356828090,67572.5,67570.0,BTC-28JUN24,67574.40,67570.0,66937.5,68650.0,NaN,NaN
3,72865.00,1568440.0,1568440.0,USD,66436.33,1713902486446,BTC,21.475140,-0.0240,69430240,72895.0,72835.0,BTC-28MAR25,72864.18,72857.5,72162.5,73682.5,NaN,NaN
4,66498.75,8524860.0,8524860.0,USD,66436.33,1713902486446,BTC,127.898725,-0.2288,17431730,66500.0,66497.5,BTC-3MAY24,66497.06,66510.0,65917.5,67367.5,NaN,NaN
5,66430.00,18720190.0,18720190.0,USD,66436.33,1713902486446,BTC,281.134934,-0.1690,285439030,66432.5,66427.5,BTC-26APR24,66435.32,66460.0,65835.0,67272.5,NaN,NaN
6,69521.25,14186140.0,14186140.0,USD,66436.33,1713902486446,BTC,203.944851,-0.1329,125642410,69522.5,69520.0,BTC-27SEP24,69527.46,69522.5,68847.5,70535.0,NaN,NaN
7,71251.25,12663270.0,12663270.0,USD,66436.33,1713902486446,BTC,176.730572,0.0632,206105440,71252.5,71250.0,BTC-27DEC24,71259.86,71260.0,70450.0,72100.0,NaN,NaN


# Structured query language (SQL)

[SQL (Structured Query Language)](https://en.wikipedia.org/wiki/SQL) is used for managing and manipulating [relational](https://en.wikipedia.org/wiki/Relational_model) databases. We will explore how to use SQL in Python using Pandas and SQLAlchemy. SQLAlchemy is a SQL toolkit and Object-Relational Mapping (ORM) library that allows Python programs to interact with databases using SQL. There exists several implementations of SQL, we will use [SQLite](https://www.sqlite.org/index.html) as a simple and lightweight example that runs out of the box with our Anaconda distribution.

Please note that this is not an SQL class so we will just scratch the surface. If you have never used SQL before, there exists plenty tutorials on the internet - also consider the links below.

See:
* https://www.sqlite.org/index.html
* https://www.sqlalchemy.org/

In [8]:
# Connect to the database
engine = create_engine('sqlite:///slides_pt5.db')

# Write data to the database
market_data.to_sql("market_data", engine, if_exists="replace", index=False)

8

At ots heart, every SQl database is a collection of tables. The code example above connects to or creates (if it doesn't exist) a database called `slides_pt5.db` and saves `market_data` into a table of the same name. 

We can now access the data using a so called SQL query. Think of a query as a filter that selects data from one or multiple tables inside a database. Here are a few code examples that collect data from a table called 'market_data' inside slides_pt5.db.

In [9]:
# Load all data
df = pd.read_sql("""SELECT * FROM market_data""", engine)

display(df)

,mid_price,volume_notional,volume_usd,quote_currency,estimated_delivery_price,creation_timestamp,base_currency,volume,price_change,open_interest,ask_price,bid_price,instrument_name,mark_price,last,low,high,funding_8h,current_funding
0,66973.75,7111840.0,7111840.0,USD,66436.33,1713902486446,BTC,105.888626,-0.1192,60707710,66982.5,66965.0,BTC-31MAY24,66980.60,67032.5,66497.5,67865.0,NaN,NaN
1,66414.25,368904940.0,368904940.0,USD,66436.33,1713902486446,BTC,5544.539464,-0.1691,775504490,66414.5,66414.0,BTC-PERPETUAL,66422.22,66414.0,65827.0,67268.0,-0.000011,0.0
2,67571.25,27468090.0,27468090.0,USD,66436.33,1713902486446,BTC,405.072827,-0.1551,356828090,67572.5,67570.0,BTC-28JUN24,67574.40,67570.0,66937.5,68650.0,NaN,NaN
3,72865.00,1568440.0,1568440.0,USD,66436.33,1713902486446,BTC,21.475140,-0.0240,69430240,72895.0,72835.0,BTC-28MAR25,72864.18,72857.5,72162.5,73682.5,NaN,NaN
4,66498.75,8524860.0,8524860.0,USD,66436.33,1713902486446,BTC,127.898725,-0.2288,17431730,66500.0,66497.5,BTC-3MAY24,66497.06,66510.0,65917.5,67367.5,NaN,NaN
5,66430.00,18720190.0,18720190.0,USD,66436.33,1713902486446,BTC,281.134934,-0.1690,285439030,66432.5,66427.5,BTC-26APR24,66435.32,66460.0,65835.0,67272.5,NaN,NaN
6,69521.25,14186140.0,14186140.0,USD,66436.33,1713902486446,BTC,203.944851,-0.1329,125642410,69522.5,69520.0,BTC-27SEP24,69527.46,69522.5,68847.5,70535.0,NaN,NaN
7,71251.25,12663270.0,12663270.0,USD,66436.33,1713902486446,BTC,176.730572,0.0632,206105440,71252.5,71250.0,BTC-27DEC24,71259.86,71260.0,70450.0,72100.0,NaN,NaN


In [10]:
# Select data based on some condition
df = pd.read_sql("""
    SELECT
        *
    FROM market_data
    WHERE open_interest > 100000000
    """, engine)

display(df)

,mid_price,volume_notional,volume_usd,quote_currency,estimated_delivery_price,creation_timestamp,base_currency,volume,price_change,open_interest,ask_price,bid_price,instrument_name,mark_price,last,low,high,funding_8h,current_funding
0,66414.25,368904940.0,368904940.0,USD,66436.33,1713902486446,BTC,5544.539464,-0.1691,775504490,66414.5,66414.0,BTC-PERPETUAL,66422.22,66414.0,65827.0,67268.0,-0.000011,0.0
1,67571.25,27468090.0,27468090.0,USD,66436.33,1713902486446,BTC,405.072827,-0.1551,356828090,67572.5,67570.0,BTC-28JUN24,67574.40,67570.0,66937.5,68650.0,NaN,NaN
2,66430.00,18720190.0,18720190.0,USD,66436.33,1713902486446,BTC,281.134934,-0.1690,285439030,66432.5,66427.5,BTC-26APR24,66435.32,66460.0,65835.0,67272.5,NaN,NaN
3,69521.25,14186140.0,14186140.0,USD,66436.33,1713902486446,BTC,203.944851,-0.1329,125642410,69522.5,69520.0,BTC-27SEP24,69527.46,69522.5,68847.5,70535.0,NaN,NaN
4,71251.25,12663270.0,12663270.0,USD,66436.33,1713902486446,BTC,176.730572,0.0632,206105440,71252.5,71250.0,BTC-27DEC24,71259.86,71260.0,70450.0,72100.0,NaN,NaN


In [11]:
# Select specific columns and (optionally) rename columns
df = pd.read_sql("""
    SELECT
        mid_price,
        high,
        low as low_price,  -- renamed column
        last as last_price,  -- renamed column
        instrument_name as ticker  -- renamed column
    FROM market_data
    WHERE open_interest > 100000000
    """, engine)

display(df)

,mid_price,high,low_price,last_price,ticker
0,66414.25,67268.0,65827.0,66414.0,BTC-PERPETUAL
1,67571.25,68650.0,66937.5,67570.0,BTC-28JUN24
2,66430.00,67272.5,65835.0,66460.0,BTC-26APR24
3,69521.25,70535.0,68847.5,69522.5,BTC-27SEP24
4,71251.25,72100.0,70450.0,71260.0,BTC-27DEC24


This is similar to import functions like `pd.read_csv()`, however, gives us a lot of flexibility in filtering the data. Also, we do not have to load the entire data into [memory (RAM)](https://www.sqlite.org/malloc.html), which allows us to handle very large datasets. If the database is stored on a server, such as [AWS-RDS](https://aws.amazon.com/rds/), the query uses the server's resources before returning only the filtered subset of the data.

# Large file storage

## .gitignore

As we are talking about storing data, it makes sense to re-visit Git and GitHub. Please note that there are **size limits on the files and the overall repository [(see HERE)](https://docs.github.com/en/repositories/working-with-files/managing-large-files/about-large-files-on-github)**. Therefore, in general, we do not want to use Git/GitHub as a storage for (large) data files or databases. In order to ensure that we do not accidentally commit files that are not supposed to be versioned, we exclude them using something called a ".gitignore" file.

See:
* https://git-scm.com/docs/gitignore

The .gitignore file is a configuration file used by Git to determine which files and directories should be ignored or excluded from being tracked. It allows you to specify patterns that match certain files or directories, indicating that Git should not consider them for version control. 

The main purpose of the ".gitignore" file is to exclude files that are generated during the development process, contain sensitive information, or are not necessary for the project's functioning. For example, build artifacts, log files, temporary files, and configuration files specific to individual developers or their environments can be safely ignored.

It's worth noting that the ".gitignore" file can be committed to the repository, allowing it to be shared among team members. However, if a file is already being tracked by Git before it's added to the ".gitignore" file, it will continue to be tracked even if it matches the specified patterns. In such cases, you need to explicitly remove the file from Git using the "git rm" command.

If you completed  homework/01_setup.ipynb, you should already have a .gitignore file inside your repository. The first few lines of the .gitignore file that I currently use for this class look like this:

```Bash
# Lecture specific
*.db

# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*$py.class

# C extensions
*.so

# Distribution / packaging
.Python
build/
develop-eggs/
dist/

# ...
```

## Alternative data storage

Besides data bases and APIs, it can be efficient to store large(r) files at dedicated storage providers, such as Dropbox. A file shared with dropbox (link) can be directly imported into python (make sure that `?dl=1`), e.g.:

In [5]:
df = pd.read_csv('https://www.dropbox.com/scl/fi/xu7ho3ogqhqpv5pricsfb/02_python_data.csv?rlkey=rds3rukek3b14glesbrfsh1ts&dl=1')
display(df)

,Date,.GDAXI,SAPG.DE,SIEGn.DE,DTEGn.DE,AIRG.DE,ALVG.DE,MBGn.DE,VOWG_p.DE,BMWG.DE,...,CONG.DE,HEIG.DE,MTXGn.DE,CBKG.DE,RHMG.DE,BNRGn.DE,ZALG.DE,QIA.DE,PSHG_p.DE,1COV.DE
0,2022-04-29,14097.88,97.026932,117.78,17.570,105.50,216.20,67.10,148.94,78.51,...,66.02,55.26,193.65,6.282,214.8,73.94,37.83,44.04,79.54,41.29
1,2022-05-02,13939.07,95.982669,115.12,17.290,103.96,215.00,62.22,147.22,77.64,...,62.46,54.74,187.90,6.204,211.6,72.30,37.74,44.01,77.48,41.41
2,2022-05-03,14039.47,94.729554,114.56,17.376,106.44,217.00,62.88,150.20,79.98,...,64.32,55.22,197.10,6.412,222.4,72.18,38.07,44.06,79.58,39.40
3,2022-05-04,13970.82,92.979171,115.78,17.530,102.68,213.55,62.33,148.52,79.47,...,63.30,55.42,192.30,6.310,213.1,72.48,37.15,43.34,78.32,40.20
4,2022-05-05,13902.52,94.699718,116.58,17.592,108.96,199.76,61.00,147.18,78.03,...,60.54,54.18,189.95,6.188,211.0,71.34,33.21,44.05,77.78,38.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2023-04-12,15703.60,116.620000,144.82,22.400,126.20,218.40,69.34,125.52,102.68,...,65.70,67.22,229.60,9.720,263.1,69.12,36.07,42.39,52.60,36.62
246,2023-04-13,15729.46,115.960000,144.28,22.305,125.90,219.10,69.79,126.08,102.32,...,66.18,67.94,231.90,9.806,265.4,69.28,36.99,42.40,52.50,35.86
247,2023-04-14,15807.50,115.960000,146.16,22.215,126.16,219.10,70.41,127.70,103.68,...,68.38,69.42,228.90,10.360,267.2,69.96,37.62,42.33,53.46,36.77
248,2023-04-17,15789.53,116.360000,147.06,22.345,128.10,216.50,69.25,126.90,103.20,...,69.10,70.18,234.60,10.000,267.0,70.48,37.31,42.06,52.54,37.14


For the exchange of very large files, one might consider the .feather file standard. A .feather file is a binary file format for storing data frames. It is part of the Apache Arrow project and was developed by Wes McKinney and Hadley Wickham to support the exchange of datasets between Python and R. The format is designed to be fast, lightweight, and easy to use, and it provides full support for data frame APIs.

Key features of the Feather format include:

* Language Agnostic: Feather uses the Apache Arrow columnar memory specification to represent binary data on disk. This makes it easy to read the data into popular data analysis languages like Python (pandas), R, and others.
* Speed: Feather efficiently uses the available system resources (like multi-core CPUs and fast disk drives) to enable high-speed read and write operations. Reading from and writing to Feather files is typically faster than with CSV or other text-based formats because data is stored in a binary format.
* No Data Copy: Feather stores data in a way that's nearly identical to its in-memory representation, which means there's no need to copy the data when reading it from disk. This also means that Feather files can be memory-mapped, which is a technique that allows programs to access files on disk as if they were part of the program's main memory.
* Metadata Compatibility: Feather files can hold all kinds of metadata, which is important in statistical data sets. It can store column names, types, and other important attributes.

Please note that Feather was not designed to be a long-term storage format. So while it's great for exchanging data between tools that understand the format, it might not be ideal for storing data for many years.

In [ ]:
df = pd.read_feather('https://www.dropbox.com/s/77fp9xuvy1xby7r/02_python_data.feather?dl=1')
display(df)

,Date,.GDAXI,SAPG.DE,SIEGn.DE,DTEGn.DE,AIRG.DE,ALVG.DE,MBGn.DE,VOWG_p.DE,BMWG.DE,...,CONG.DE,HEIG.DE,MTXGn.DE,CBKG.DE,RHMG.DE,BNRGn.DE,ZALG.DE,QIA.DE,PSHG_p.DE,1COV.DE
0,2022-04-29,14097.88,97.026932,117.78,17.570,105.50,216.20,67.10,148.94,78.51,...,66.02,55.26,193.65,6.282,214.8,73.94,37.83,44.04,79.54,41.29
1,2022-05-02,13939.07,95.982669,115.12,17.290,103.96,215.00,62.22,147.22,77.64,...,62.46,54.74,187.90,6.204,211.6,72.30,37.74,44.01,77.48,41.41
2,2022-05-03,14039.47,94.729554,114.56,17.376,106.44,217.00,62.88,150.20,79.98,...,64.32,55.22,197.10,6.412,222.4,72.18,38.07,44.06,79.58,39.40
3,2022-05-04,13970.82,92.979171,115.78,17.530,102.68,213.55,62.33,148.52,79.47,...,63.30,55.42,192.30,6.310,213.1,72.48,37.15,43.34,78.32,40.20
4,2022-05-05,13902.52,94.699718,116.58,17.592,108.96,199.76,61.00,147.18,78.03,...,60.54,54.18,189.95,6.188,211.0,71.34,33.21,44.05,77.78,38.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2023-04-12,15703.60,116.620000,144.82,22.400,126.20,218.40,69.34,125.52,102.68,...,65.70,67.22,229.60,9.720,263.1,69.12,36.07,42.39,52.60,36.62
246,2023-04-13,15729.46,115.960000,144.28,22.305,125.90,219.10,69.79,126.08,102.32,...,66.18,67.94,231.90,9.806,265.4,69.28,36.99,42.40,52.50,35.86
247,2023-04-14,15807.50,115.960000,146.16,22.215,126.16,219.10,70.41,127.70,103.68,...,68.38,69.42,228.90,10.360,267.2,69.96,37.62,42.33,53.46,36.77
248,2023-04-17,15789.53,116.360000,147.06,22.345,128.10,216.50,69.25,126.90,103.20,...,69.10,70.18,234.60,10.000,267.0,70.48,37.31,42.06,52.54,37.14


# Virtual environments

The Python programming language offers an enormous variety of libraries and modules that allow developers to implement diverse functionalities. Often, each Python project requires a specific set of libraries and may even depend on certain versions of these libraries. However, installing and managing different versions of the same library globally on your system for different projects can lead to conflicts and incompatibility issues. This is where virtual environments come in handy.

In Python, a virtual environment is a self-contained directory that encapsulates a specific Python interpreter and its associated packages and dependencies. It allows you to create isolated environments with their own set of installed libraries, independent of the globally installed packages on your system.

Virtual environments are useful when you're working on multiple projects or collaborating with others, as they help ensure that each project has its own independent set of dependencies. This way, you can avoid conflicts between different versions of packages required by different projects.

To create a virtual environment in Python, you can use the built-in module called venv (available in Python 3.3 and higher) or third-party tools like virtualenv or conda. 

Environments are particularly useful when working on different projects that require different versions of packages or when collaborating with others who have different package requirements. They provide a reliable and reproducible way to manage and isolate your project's dependencies. **This might be relevant for the code submitted for this course.**

## Conda environments

Conda environments are isolated environments created by the Conda package manager. Conda is a popular package management system used primarily in the Python ecosystem, although it can also manage packages for other programming languages. In my humble opinion:
* Virtual environments are good programming practice!
* Conda is more convenient if you already use Anaconda.
* Conda can do (install) a bit more than PIP [(see HERE)](https://stackoverflow.com/questions/20994716/what-is-the-difference-between-pip-and-conda).
* Using Conda and PIP to install packages into the same environnement can get you into trouble.

See:
* https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html

To create a virtual environment

```Bash
conda create -n myenv python=3.10 pip numpy scipy pandas spyder # Make sure you install pip directly here!
conda activate myenv
# Install additional packages, e.g.:
conda install sqalchemy
# ...
conda deactivate
```

Use your newly created environment

```Bash
conda activate myenv
# Have fun!
conda deactivate
```

Now the whole idea was to be able to provide a reliable and reproducible way to manage and isolate your project's dependencies. So lets save our environment:

```Bash
conda activate myenv
conda env export > requirements.yml  # Create a list of packages that is Conda compatible
pip list --format=freeze > requirements.txt  # Create a list of packages that is PIP compatible
conda deactivate
```

This should have created two files (requirements.yml and requirements.txt). We can use these files, for example on a different system, to reproduce the environment:

```Bash
conda env create -f requirements.yml  # Install for the first time from .yml
```

We are therefore able to share the exact and hopefully tested environment, such that a third person can reproduce our exact results.

## Venv environments

The venv module provides a convenient way to create virtual environments, which are self-contained environments with their own separate set of Python packages installed in dedicated directories. A virtual environment is built upon an existing Python installation, referred to as the "base" Python, and it can be configured to be isolated from the packages installed in the base environment. This isolation ensures that only the packages explicitly installed within the virtual environment are accessible.

When operating within a virtual environment, popular installation tools like pip automatically install Python packages into the virtual environment without requiring explicit instructions to do so. This streamlined process allows for seamless package management within the virtual environment.

See:
* https://docs.python.org/3/library/venv.html

#  Server infrastructure

Many projects, especially those involving scraping, require us to regularly acquire and store data. It is therefore inefficient to manually start the process or to run it on a machine that is not permanently connected to the internet. Luckily there are several server based options available that allow is to continuously run our software. 

One of the world's largest server providers is [Amazon Web Services (AWS)](https://aws.amazon.com/). A cheaper option, with a free plan for academic purposes is [PythonAnywhere](https://www.pythonanywhere.com/), which we will use as an example in this section.

### Recurring tasks

Let's have a look at two programming paradigms that allow us to run tasks recurring tasks continuously:

1. Infinite loops
2. Tasks scheduled by the operating system

Let us start by defining a task that does is robust to exceptions. When not continuously monitoring a task, it is good practice to create a .log file that captures information on the execution, which is extremely helpful for debugging. Have a look at Python's [Logging package](https://docs.python.org/3/library/logging.html).

In [ ]:
def task():
    try:
        # The actual task goes here
        print("Trying to solve the task ...")
        raise Exception('The code will survive this exception.')
    except Exception as e:
        # One could fix or log exceptions here
        print("There was an exception, however,")
        print(e)
    finally:
        # This part is executed on every pass (exception or not)
        # Consider closing database connections here
        print("Finally ...")

task()

Trying to solve the task ...
There was an exception, however,
The code will survive this exception.
Finally ...


#### Infinite loop

Obviously, the task above runs only once. If we want to run in permanently, we have to nest it in a loop or schedule it. Here is an example for a corresponding loop:

In [ ]:
while True:
    # Execute the task ...
    task()
    # Try to be gentle to your system resources ...
    time.sleep(1)
    # Let's kill the loop ...
    raise Exception('This exception will kill the loop.')


Trying to solve the task ...
There was an exception, however,
The code will survive this exception.
Finally ...


Exception: This exception will kill the loop.

Another, and perhaps more elegant, solution is to schedule the task. This works like a charm on Unix based system and PythonAnywhere but is a major pain on Windows - so I do not even try the latter anymore.

#### Cron

On a Unix system, simply run the bash command
```Bash
crontab -e
```
[Cron](https://en.wikipedia.org/wiki/Cron) allows you any task that could also run in the command line, including any .py script.

```Bash
# ┌───────────── minute (0 - 59)
# │ ┌───────────── hour (0 - 23)
# │ │ ┌───────────── day of the month (1 - 31)
# │ │ │ ┌───────────── month (1 - 12)
# │ │ │ │ ┌───────────── day of the week (0 - 6) (Sunday to Saturday;
# │ │ │ │ │                                       7 is also Sunday on some systems)
# │ │ │ │ │
# │ │ │ │ │
# * * * * * <command to execute>
```

#### PythonAnywhere

It get's even easier with a service like [PythonAnywhere](https://www.pythonanywhere.com/) that wraps task scheduling into a web interface, see: 

* https://help.pythonanywhere.com/pages/
* https://www.pythonanywhere.com/user/[yourusername]/tasks_tab/

By the way, you can also run web applications, such as live reporting of a trading algorithm :) in PythonAnywhere, see: 

* https://help.pythonanywhere.com/pages/FollowingTheDjangoTutorial/
* https://github.com/cafawo/basicdjango
* https://thecvx.com/

Reporting your results with sth. like that would impress me plenty, however, is not a must have.

## Logging


In Python, [logging](https://docs.python.org/3/library/logging.html) is a standard library module used to track events or issues that occur when software runs. The logging module offers a full-featured and flexible logging system. At its simplest, log messages are sent to a log file or the console. However, you can also configure loggers to send messages to various other locations, including network sockets.

Logging provides several benefits over using print statements:

1. It provides a way to categorize and prioritize your log messages. This is done using log levels like DEBUG, INFO, WARNING, ERROR, CRITICAL.

2. It can write log messages to any location. With print statements, the output goes to the standard output (i.e., the console), but logging can write to files, sockets, HTTP GET/POST locations, email, or any other kind of data stream.

3. It is easy to differentiate regular output from debugging information. This is because logging provides a way to "turn off" or "turn on" messages of various importance.

4. It can provide more detailed context for each message, like where and when a problem occurred, what function or module it occurred in, etc.

First, lets set up the logger ...


In [ ]:
# Logging specific
import logging
import time
logging.basicConfig(format='# %(asctime)s %(message)s',
                    filename='slides_pt5.log',
                    level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler())
logging.Formatter.converter = time.gmtime

With the logger in place, we can start to log stuff ...

In [ ]:
# Log some messages
logging.debug('This is a debug message')
logging.info('This is an informational message')
logging.warning('Careful! Something does not look right')
logging.error('An error occurred')
logging.critical('Critical failure')

Careful! Something does not look right
An error occurred
Critical failure


Logging can also capture exceptions that occur within a `try` statement.

In [ ]:
try:
    raise Exception("Sth. went wrong!")
except Exception as e:
    logging.exception(e)

Sth. went wrong!
Traceback (most recent call last):
  File "C:\Users\nb\AppData\Local\Temp\ipykernel_12928\2382030352.py", line 2, in <module>
    raise Exception("Sth. went wrong!")
Exception: Sth. went wrong!
